In [1]:

import numpy as np
import pandas as pd
import math
import seaborn as sns
from sklearn import preprocessing
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split
import datetime
from sklearn.model_selection import GridSearchCV

In [2]:
test = pd.read_csv("../input/tp2grupo3/test.csv")
train = pd.read_csv("../input/tp2grupo3/train.csv")

train.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0


In [3]:
train["tipodepropiedad"].fillna("Vacio", inplace=True)
test["tipodepropiedad"].fillna("Vacio", inplace=True)

train["provincia"].fillna("Vacio", inplace=True)
test["provincia"].fillna("Vacio", inplace=True)

train["ciudad"].fillna("Vacio", inplace=True)
test["ciudad"].fillna("Vacio", inplace=True)

train.fillna({'habitaciones': 0.0}, inplace = True)
train.fillna({'garages': 0.0}, inplace = True)
train.fillna({'banos': 0.0}, inplace = True)
train.fillna({'metroscubiertos': 0.0}, inplace = True)
train.fillna({'metrostotales': 0.0}, inplace = True)


test.fillna({'habitaciones': 0.0}, inplace = True)
test.fillna({'garages': 0.0}, inplace = True)
test.fillna({'banos': 0.0}, inplace = True)
test.fillna({'metroscubiertos': 0.0}, inplace = True)
test.fillna({'metrostotales': 0.0}, inplace = True)

In [4]:
meanPrecios = train.groupby(['provincia', "ciudad"])
meanPrecios = meanPrecios.agg({
         'precio': "mean"
    })


meanPrecios.head(100)

precio
provincia      ciudad                      
Aguascalientes Aguascalientes  1.545951e+06
               Asientos        1.954870e+06
               Calvillo        1.653000e+06
               Cosío           8.866667e+05
               El Llano        1.333333e+06
...                                     ...
Colima         Colima          1.598735e+06
               Comala          3.097800e+06
               Coquimatlán     3.500000e+05
               Cuauhtémoc      1.284824e+06
               Ixtlahuacán     1.150000e+06

[100 rows x 1 columns]

In [5]:
meanPreciosProvincia = train.groupby(['provincia'])
meanPreciosProvincia = meanPreciosProvincia.agg({
         'precio': "mean"
    })
meanPreciosProvincia.head(100)

,precio
provincia,
Aguascalientes,1.477189e+06
Baja California Norte,7.674418e+05
Baja California Sur,1.216453e+06
Campeche,1.570918e+06
Chiapas,1.924126e+06
Chihuahua,1.744531e+06
Coahuila,1.505305e+06
Colima,1.531380e+06
Distrito Federal,3.471705e+06


In [6]:
def getPrecioPromedio(row):
    try: 
        return meanPrecios.loc[row["provincia"]].loc[row["ciudad"]].precio
    except KeyError:
        return meanPreciosProvincia.loc[row["provincia"]].precio

In [7]:
def agregarPrecioPromedio(dfReceived):
    dfReceived["precio_promedio_ubicacion"] = dfReceived.apply(lambda row: getPrecioPromedio(row), axis = 1)
    return dfReceived

In [8]:
#train = agregarPrecioPromedio(train)
#test = agregarPrecioPromedio(test)

In [9]:
def agregarFactorUbicacion(dfReceived, max):
    dfReceived["factor_ubicacion"] = dfReceived.apply(lambda row: 100*(row["precio_promedio_ubicacion"]/max), axis=1)

In [10]:
#max = train["precio_promedio_ubicacion"].max()
#agregarFactorUbicacion(train, max)
#agregarFactorUbicacion(test, max)

In [11]:
test['fecha'] = pd.to_datetime(test['fecha'])
test['year'] = test['fecha'].dt.year
test['month']= test['fecha'].dt.month

train['fecha'] = pd.to_datetime(train['fecha'])
train['year'] = train['fecha'].dt.year
train['month']= train['fecha'].dt.month

In [12]:
train = train.drop(['fecha','titulo','descripcion','direccion','lat','lng'], axis=1)
test = test.drop(['fecha','titulo','descripcion','direccion','lat','lng'], axis=1)
train = train.fillna(value = {'provincia': 0, 'tipodepropiedad': 0, 'ciudad':0})
test = test.fillna(value = {'provincia': 0, 'tipodepropiedad': 0, 'ciudad':0})

ciudades_en_train = train.ciudad.unique()
ciudades_en_test = test.ciudad.unique()

tipodepropiedad_en_train = train.tipodepropiedad.unique()
tipodepropiedad_en_test = test.tipodepropiedad.unique()

provincia_en_train = train.provincia.unique()
provincia_en_test = test.provincia.unique()

In [13]:
train = pd.get_dummies(train, dummy_na=True)
test = pd.get_dummies(test, dummy_na=True)

In [ ]:
train.shape


In [14]:
X = train.drop("precio", axis=1)
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [15]:
xgb1 = XGBRegressor(verbose = True,tree_method = 'gpu_hist', gpu_id = 0,reg_alpha=0.00006,seed=24)

#Ir cambiando parametros para ver como functiona

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.04], 
              'max_depth': [11], 
              'min_child_weight': [9], 
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7], 
              'n_estimators': [1500],  
            'gamma': [0]} 

#0.7876343276013036 con 700 estimadores y 9 depth con fechas parseadas
#0.7903438420940652 con 700 estimadores y 9 depth
#0.7919269007179323 con 600 estimadores y 11 depth
#0.8022990816922799 con 700 estimadores , 9 depth, 5 min_child_weight
#0.8037024620856591 con 600 estimadores , 11 depth, 5 min_child_weight
#0.8037805503151154 con 600 estimadores, 11 depth, 9 min_child_weight
#0.805378985255433 con 1000 estimadores, 11 depth, 9 min_child_weight
#0.8056895246147905 con 1500 estimadores, 11 depth, 9 min_child_weight
#0.8062374523234128 con IDEM pero alpha reg_alpha=0.00006
#0.8075607001267892 con 0.01, 3000 estimadores, alpha=0.00006

#0.8033438399574073 con factor_localizacion
#0.8049173733076324 con learning rate 0.03
#0.8061640249880834 con learning rate 0.02
#0.8062457596721254 con learning rate 0.02, 1600 estimadores
#0.8063147276283286 con learning rate 0.02, 2000 estimadores
#0.8069990950451683 con learning rate 0.01, 3000 estimadores

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

#xgb_grid.fit(X_train,y_train,verbose=True)
xgb_grid.fit(X,y,verbose=True)


print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   2 | elapsed:  9.0min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   2 out of   2 | elapsed:  9.0min finished
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


0.8075607001267892
{'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 11, 'min_child_weight': 9, 'n_estimators': 3000, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


In [ ]:
#from xgboost import plot_importance
#from matplotlib import pyplot

#plot_importance(xgb_grid, max_num_features = 10)
#pyplot.show()

In [16]:
preds2 = xgb_grid.predict(X_test)


In [17]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, preds2)

446286.15675172524

In [18]:
for ciudad in ciudades_en_train:
    if ciudad not in ciudades_en_test:
        test['ciudad_'+ str(ciudad)] = 0
        
for provincia in provincia_en_train:
    if provincia not in provincia_en_test:
        test['provincia_'+ str(provincia)] = 0
        
for tipodepropiedad in tipodepropiedad_en_train:
    if tipodepropiedad not in tipodepropiedad_en_test:
        test['tipodepropiedad_'+ str(tipodepropiedad)] = 0

In [19]:
test = test[list(X.columns.values)]
preds = xgb_grid.predict(test)

In [20]:
res = pd.DataFrame(preds, index=test.index, columns=['precio'])

In [21]:
res2 = res
res2['id'] = test['id']
res2 = res.reset_index(drop=True)
res2.head()
res2.set_index('id', inplace = True)
#res.columns = ['id', 'target']
#res.set_index('id', inplace = True)
res2.rename(columns={'precio':'target'}, inplace=True)
res2.head()


#res2.to_csv("/kaggle/working/submission_xgboost_city_4.csv", header=True)

,target
id,
4941,6.927870e+06
51775,9.429387e+05
115253,2.440284e+06
299321,1.463405e+06
173570,5.755709e+05


In [22]:
res2.to_csv("/kaggle/working/submission_xgboost_14.csv", header=True)